# Predicción con Transformers

* Diego Elián Rodríguez Cantú A00829925
* Oscar Antonio Banderas Álvarez A01568492
* Taurino López González A01284076
* Ivan Karlo González Barreda A01025331
* Axel Antonio Maldonado Del Bosque A00830422
* Elias Garza Valdés A01284041


In [1]:
import os
import numpy as np
import pandas as pd

from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split


In [2]:
def load_data(filename):
    data = pd.read_csv(filename)
    temperature = data['T (degC)'].values  # Tomar solo la columna de temperatura
    return temperature

data = load_data("jena_climate_2009_2016.csv")

In [12]:
data = pd.read_csv("jena_climate_2009_2016.csv")
data

,Date Time,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),rho (g/m**3),wv (m/s),max. wv (m/s),wd (deg)
0,01.01.2009 00:10:00,996.52,-8.02,265.40,-8.90,93.30,3.33,3.11,0.22,1.94,3.12,1307.75,1.03,1.75,152.3
1,01.01.2009 00:20:00,996.57,-8.41,265.01,-9.28,93.40,3.23,3.02,0.21,1.89,3.03,1309.80,0.72,1.50,136.1
2,01.01.2009 00:30:00,996.53,-8.51,264.91,-9.31,93.90,3.21,3.01,0.20,1.88,3.02,1310.24,0.19,0.63,171.6
3,01.01.2009 00:40:00,996.51,-8.31,265.12,-9.07,94.20,3.26,3.07,0.19,1.92,3.08,1309.19,0.34,0.50,198.0
4,01.01.2009 00:50:00,996.51,-8.27,265.15,-9.04,94.10,3.27,3.08,0.19,1.92,3.09,1309.00,0.32,0.63,214.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420446,31.12.2016 23:20:00,1000.07,-4.05,269.10,-8.13,73.10,4.52,3.30,1.22,2.06,3.30,1292.98,0.67,1.52,240.0
420447,31.12.2016 23:30:00,999.93,-3.35,269.81,-8.06,69.71,4.77,3.32,1.44,2.07,3.32,1289.44,1.14,1.92,234.3
420448,31.12.2016 23:40:00,999.82,-3.16,270.01,-8.21,67.91,4.84,3.28,1.55,2.05,3.28,1288.39,1.08,2.00,215.2
420449,31.12.2016 23:50:00,999.81,-4.23,268.94,-8.53,71.80,4.46,3.20,1.26,1.99,3.20,1293.56,1.49,2.16,225.8


In [11]:
data

array([-8.02, -8.41, -8.51, ..., -3.16, -4.23, -4.82])

In [3]:
mean = data.mean(axis=0)
std = data.std(axis=0)
data_normalized = (data - mean) / std

train_data, test_data = train_test_split(data_normalized, test_size=0.2)

In [4]:
train_data

array([-0.07343195, -1.97402758, -1.36859112, ..., -0.91273308,
       -1.13710071, -0.62426041])

In [5]:
x_train = train_data.reshape((train_data.shape[0], 1, 1))
x_test = test_data.reshape((test_data.shape[0], 1, 1))

n_classes = 1

In [6]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Attention y Normalización
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs

    # Parte Feed Forward
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(res)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    return x + res

In [7]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Attention y Normalización
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs

    # Parte Feed Forward
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(res)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    return x + res

In [8]:
def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_last")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(n_classes)(x)  # Una neurona de salida para la predicción de temperatura
    return keras.Model(inputs, outputs)

In [9]:
input_shape = x_train.shape[1:]

model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
)

In [10]:
model.compile(
    loss="mse",  # Usar MSE para regresión
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics=["mae"],  # Métrica de error absoluto medio
)
model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

model.fit(
    x_train,
    x_train,  # Usamos x_train como target, ya que estamos tratando de predecir la temperatura
    validation_split = 0.2,
    epochs = 25,
    batch_size = 64,
    callbacks = callbacks,
)
mse, mae = model.evaluate(x_test, x_test, verbose=1)
print("Mean Squared Error on Test Data:", mse)
print("Mean Absolute Error on Test Data:", mae)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1, 1)]       0           []                               
                                                                                                  
 multi_head_attention (MultiHea  (None, 1, 1)        7169        ['input_1[0][0]',                
 dAttention)                                                      'input_1[0][0]']                
                                                                                                  
 dropout (Dropout)              (None, 1, 1)         0           ['multi_head_attention[0][0]']   
                                                                                                  
 layer_normalization (LayerNorm  (None, 1, 1)        2           ['dropout[0][0]']            

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

# Predicciones del modelo en los datos de entrenamiento y prueba
train_predictions = model.predict(x_train)
test_predictions = model.predict(x_test)

# Deshacer la normalización de los datos
train_predictions = (train_predictions * std) + mean
test_predictions = (test_predictions * std) + mean

# Deshacer la normalización de los datos de prueba (inputs)
x_test_original = (x_test * std) + mean

# Graficar resultados
plt.figure(figsize=(12, 6))

# Graficar datos de entrenamiento
plt.plot(x_train.squeeze(), label='Training Data', color='blue')

# Graficar datos de prueba
plt.plot(range(len(x_train), len(x_train) + len(x_test)), x_test_original.squeeze(), label='Test Data (Actual)', color='green')

# Graficar predicciones en datos de entrenamiento
plt.plot(range(len(train_predictions)), train_predictions.squeeze(), label='Training Predictions', color='red')

# Graficar predicciones en datos de prueba
plt.plot(range(len(x_train), len(x_train) + len(test_predictions)), test_predictions.squeeze(), label='Test Predictions', color='orange')

plt.title('Temperature Prediction')
plt.xlabel('Time')
plt.ylabel('Temperature (degC)')
plt.legend()
plt.show()


: 